# Forward Thinking Comparison
##### Sean Wade

In [1]:
from forwardThinking.datasets import load_mnist
from forwardThinking.models import PassForwardThinking, ForwardThinking, DNN
import numpy as np
from itertools import product

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

from IPython.core.display import HTML
import urllib2
HTML(open("./jupyter.css", "r").read())

Using TensorFlow backend.


In [2]:
def single_pixel_tr(X, y, img_size=(28,28)):
        n_samples, dim_data = X.shape
        X_out = np.empty((n_samples*9, dim_data))
        y_out = np.zeros(n_samples*9)
        for k in xrange(10):
            k_image = X[k,:].reshape(img_size)
            for i,j in product([-1,0,1], [-1,0,1]):
                i_row = 3*(i+1)
                image = np.zeros((img_size[0]+2, img_size[1]+2))
                image[1+i:1+i+img_size[0], 1+j:1+j+img_size[1]] = k_image
                X_out[9*k+i_row+j+1] = image[1:1+img_size[0],1:1+img_size[1] ].flatten()
                y_out[9*k+i_row+j+1] = y[k]
        return X_out, y_out
    
def load_mnist_aug(flatten=True, one_hot_labels=True):
    """ Load mnist dataset using keras."""
    from keras.datasets import mnist
    from keras.utils import np_utils

    (x_train, y_train_num), (x_test, y_test_num) = mnist.load_data()
    (x_train, y_train_num) = single_pixel_tr(x_train.reshape(60000,784), y_train_num, img_size=(28,28))
    if flatten:
        flatten_shape = np.prod(x_train.shape[1:])
        x_train = x_train.reshape(x_train.shape[0], flatten_shape).astype('float32')
        x_test = x_test.reshape(x_test.shape[0], flatten_shape).astype('float32')
    else:
        x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')
        x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32')

    x_train /= 255
    x_test /= 255
    y_train = np_utils.to_categorical(y_train_num, 10)
    y_test = np_utils.to_categorical(y_test_num, 10)

    if not one_hot_labels:
        y_train = y_train_num
        y_test = y_test_num
    
    return x_train, y_train, x_test, y_test

In [3]:
%load_ext autoreload
%autoreload 2

### Helper Functions

In [4]:
def plot_acc_loss(acc, loss, val_acc=None, val_loss=None):
    plt.figure(figsize=(15,7))
    plt.subplot(121)
    plt.title('Accuracy')
    plt.plot(acc)
    if val_acc != None:
        plt.plot(val_acc)
    plt.subplot(122)
    plt.title('Loss')
    plt.plot(loss)
    if val_loss != None:
        plt.plot(val_loss)
    plt.tight_layout()
    plt.show()

In [5]:
def plot_all(model_list):
    plt.figure(figsize=(15,15))
    plt.subplot(221)
    plt.title('Train Accuracy')
    plt.subplot(222)
    plt.title('Test Accuracy')
    plt.subplot(223)
    plt.title('Train Loss')
    plt.subplot(224)
    plt.title('Test Loss')
    
    color = plt.cm.viridis_r(np.linspace(0,1,len(model_list)))
    for x, c in zip(model_list, color):
        name, acc, loss, val_acc, val_loss = x
        plt.subplot(221)
        plt.plot(acc, label=name, c=c)
        
        plt.subplot(222)
        plt.plot(val_acc, label=name, c=c)
        
        plt.subplot(223)
        plt.plot(loss, label=name, c=c)
        
        plt.subplot(224)
        plt.plot(val_loss, label=name, c=c)
    
    plt.legend(loc='best')
    plt.tight_layout()
    plt.savefig('/home/seanwade/Desktop/comarison.png')
    plt.show()


        
        
def make_tuple(name, acc, loss, val_acc=None, val_loss=None):
    return (name, acc, loss, val_acc, val_loss)

In [6]:
def model_summary(model):
    print "Model Name: %s" % model.summary['model_name']
    print "Model Version: %-20s" % model.summary['model_version']
    print "Training Time: %s" % model.summary['training_time']
    print "Training Acc: %s" % model.summary['accuracy'][-1]
    print "Testing Acc: %s" % model.summary['val_accuracy'][-1]

In [7]:
# Load the data
x_train, y_train, x_test, y_test = load_mnist_aug()

In [9]:
plot_list = []
plot_list.append(make_tuple('DNN', dnn.summary['accuracy'], dnn.summary['loss'],
             dnn.summary['val_accuracy'], dnn.summary['val_loss']))
plot_list.append(make_tuple('ForwardThinking', forward.summary['accuracy'], forward.summary['loss'],
             forward.summary['val_accuracy'], forward.summary['val_loss']))
plot_list.append(make_tuple('PassForward', passForward.summary['accuracy'], passForward.summary['loss'],
             passForward.summary['val_accuracy'], passForward.summary['val_loss']))
plot_list.append(make_tuple('Weight Freeze', passForwardFreeze.summary['accuracy'], passForwardFreeze.summary['loss'],
             passForwardFreeze.summary['val_accuracy'], passForwardFreeze.summary['val_loss']))

plot_all(plot_list)

NameError: name 'dnn' is not defined

## DNN

A simple DNN. No big deal.

In [ ]:
dnn = DNN([784, 100, 50, 10])
dnn.fit(x_train, y_train, x_test, y_test, epochs=30, verbose=False)

In [ ]:
plot_acc_loss(dnn.summary['accuracy'], dnn.summary['loss'],
             dnn.summary['val_accuracy'], dnn.summary['val_loss'])

In [ ]:
model_summary(dnn)

## Forward Thinking

Pass the data through a layer and into new uninitialized layer

In [ ]:
forward = ForwardThinking([784, 100, 50, 10])
forward.fit(x_train, y_train, x_test, y_test, epochs=10, verbose=False)

In [ ]:
plot_acc_loss(forward.summary['accuracy'], forward.summary['loss'],
             forward.summary['val_accuracy'], forward.summary['val_loss'])

In [ ]:
model_summary(forward)

## Pass Forward Thinking

Train layerwise, but also keep connections from old data in classification.

In [ ]:
passForward = PassForwardThinking([784, 100, 50, 10], freeze=False)
passForward.fit(x_train, y_train, x_test, y_test, epochs= 10, verbose=False)

In [ ]:
plot_acc_loss(passForward.summary['accuracy'], passForward.summary['loss'],
             passForward.summary['val_accuracy'], passForward.summary['val_loss'])

In [ ]:
model_summary(passForward)

## Pass Forward Thinking (with weight freezing)

Train layerwise like before, however freeze the connections of old weights once learned.

In [ ]:
passForwardFreeze = PassForwardThinking([784, 100, 50, 10], freeze=True)
passForwardFreeze.fit(x_train, y_train, x_test, y_test, epochs=10, verbose=False)

In [ ]:
plot_acc_loss(passForwardFreeze.summary['accuracy'], passForwardFreeze.summary['loss'],
             passForwardFreeze.summary['val_accuracy'], passForwardFreeze.summary['val_loss'])

In [ ]:
model_summary(passForwardFreeze)

## Observations

These results are on just MNIST so the gains arn't huge, however the results are pretty sweet! Pass forward approaches overcome the issues with loosing accuracy when new layers are added. Further, freezing the layers serves as a way to regularize and speed it up. Both pass forwards are consistently better than a DNN with the same archetecture. It is also cool to note we can see when layers are added in the accuracy graph. There are increases at epochs 10 and 20. 